In [5]:
import os 
import sys
import ismn
import pandas as pd
import numpy as np
import xarray as xr
from multiprocessing import Pool, cpu_count # 
from pathlib import Path
from datetime import datetime
import warnings
from ismn.interface import ISMN_Interface

In [6]:
insitu_dir = '/home/khanalp/data/ISMNsoilMoisture/'

In [7]:
# Read the data using ISMN_Interface
ds = ISMN_Interface("/home/khanalp/data/ISMNsoilMoisture/Data_separate_files_header_20140101_20251231_13107_18mx_20260208", parallel=True)

Using the existing ismn metadata in /home/khanalp/data/ISMNsoilMoisture/Data_separate_files_header_20140101_20251231_13107_18mx_20260208/python_metadata/Data_separate_files_header_20140101_20251231_13107_18mx_20260208.csv to set up ISMN_Interface. 
If there are issues with the data reader, you can remove the metadata csv file to repeat metadata collection.


In [8]:
ds

ismn.base.IsmnRoot Unzipped at /home/khanalp/data/ISMNsoilMoisture/Data_separate_files_header_20140101_20251231_13107_18mx_20260208
with Networks[Stations]:
------------------------
  AMMA-CATCH: ['Banizoumbou', 'Belefoungou-Mid', 'Belefoungou-Top', 'Nalohou-Mid', 'Nalohou-Top', 'Tondikiboro', 'Wankama'],
  ARM: ['Anthony', 'Ashton', 'Byron', 'Lamont-CF1', 'Lamont-CF2', 'MapleCity', 'Marshall', 'Medford', 'Morrison', 'Newkirk', 'Okmulgee', 'Omega', 'Pawhuska', 'Pawnee', 'Ringwood', 'Tryon', 'Tyro', 'Waukomis'],
  BDF_Saxony: ['Hilbersdorf', 'Koellitsch', 'Lippen', 'Schmorren'],
  BFG_Nw: ['BFG-Niederwerth-1-weighable'],
  BIEBRZA_S-1: ['grassland-soil-1', 'grassland-soil-2', 'grassland-soil-3', 'grassland-soil-4', 'grassland-soil-5', 'grassland-soil-6', 'grassland-soil-7', 'grassland-soil-8', 'grassland-soil-9', 'marshland-soil-11', 'marshland-soil-12', 'marshland-soil-13', 'marshland-soil-14', 'marshland-soil-15', 'marshland-soil-16', 'marshland-soil-17', 'marshland-soil-18', 'marshla

In [9]:
# 1. List networks and stations using modern ISMN interface
networks_stations = []
for network in ds.collection.networks:
    for station in ds.collection[network].stations:
        networks_stations.append({
            'network': network,
            'station': station
        })


In [10]:
df_stations = pd.DataFrame(networks_stations)
print(f"Total stations: {len(df_stations)}")
df_stations

Total stations: 2393


,network,station
0,AMMA-CATCH,Banizoumbou
1,AMMA-CATCH,Belefoungou-Mid
2,AMMA-CATCH,Belefoungou-Top
3,AMMA-CATCH,Nalohou-Mid
4,AMMA-CATCH,Nalohou-Top
...,...,...
2388,iRON,NorthstarAspenGrove
2389,iRON,NorthstarTransitionZone
2390,iRON,SkyMountain
2391,iRON,SmugglerMountain


In [21]:
# 2. Select one station to test (change these to your preferred station)
test_network = df_stations.iloc[0]['network']  # First network
test_station = df_stations.iloc[0]['station']  # First station

print(f"\n--- Testing with: {test_network} - {test_station} ---\n")
ds_test = ds[test_network][test_station].to_xarray()
ds_test # To check how it looks like. 


--- Testing with: AMMA-CATCH - Banizoumbou ---



<xarray.Dataset> Size: 2MB
Dimensions:                  (sensor: 2, date_time: 43824)
Coordinates:
  * date_time                (date_time) datetime64[ns] 351kB 2014-01-01 ... ...
Dimensions without coordinates: sensor
Data variables: (12/25)
    soil_moisture            (sensor, date_time) float64 701kB 0.0051 ... 0.0174
    soil_moisture_flag       (sensor, date_time) object 701kB 'G' 'G' ... 'G'
    soil_moisture_orig_flag  (sensor, date_time) object 701kB 'M' 'M' ... 'M'
    depth_from               (sensor) float64 16B 0.05 0.05
    depth_to                 (sensor) float64 16B 0.05 0.05
    clay_fraction            (sensor) float64 16B 5.0 5.0
    ...                       ...
    saturation               (sensor) float64 16B 0.42 0.42
    silt_fraction            (sensor) float64 16B 5.0 5.0
    station                  (sensor) <U11 88B 'Banizoumbou' 'Banizoumbou'
    timerange_from           (sensor) datetime64[ns] 16B 2014-01-01 2014-01-01
    timerange_to             (sensor) datetime64[ns] 16B 2018-12-31T23:00:00 ...
    variable                 (sensor) <U13 104B 'soil_moisture' 'soil_moisture'
Attributes:
    station_name:  Banizoumbou
    lat:           13.5325
    lon:           2.6604
    n_sensors:     2

In [12]:
# Create output directory
output_dir = Path('processed_soil_moisture')
output_dir.mkdir(exist_ok=True)

In [25]:
ds_test.attrs.get('lon', np.nan)

2.6604

To learn about ISMN data quality flag: check https://ismn.earth/en/data/flag-overview/
In short, G = good, D = Dubious, C= outside plausible range.

Inside process_station, \
1. Create a mask for good-quality measurements: soil_moisture_flag == "G".
2. Apply the mask to soil_moisture so non-G values become NaN.
3. Add a coordinate depth_group (per sensor) using depth_from so sensors can be grouped by depth.
4. Group by depth_group and take the mean across sensors (depth-average), ignoring NaNs.
5. Rename depth_group to depth.
6. Compute how many valid values each depth has over all times; drop depths that are all NaN.
7. If no depths remain, return None.
8. Resample to daily mean soil moisture (per depth).
9. Resample to a daily count of valid observations (per depth).
10.Try to convert the daily count to an integer (fallback to a float if conversion fails).
11. Mask daily soil moisture where the daily count is < 6 (those days become NaN).
12. Return an xarray.Dataset with:
13. soil_moisture (daily, filtered by count)
14. observation_count (daily count)
Add metadata attributes: network, station, latitude, and longitude.

In [4]:
# Function to process each station
def process_station(station_data, network, station):
    """
    Process a single station dataset:
    - Depth average soil_moisture where flag = 'G'
    - Drop depths with no valid data
    - Convert to daily if >= 6 valid observations per day
    """
    # Create mask where soil_moisture_flag == "G"
    mask = station_data['soil_moisture_flag'] == "G"
    
    # Apply mask to soil_moisture
    soil_moisture_masked = station_data['soil_moisture'].where(mask)
    
    # Assign depth_from as a coordinate for grouping
    soil_moisture_masked = soil_moisture_masked.assign_coords(
        depth_group=('sensor', station_data['depth_from'].values)
    )
    
    # Group by depth and average across sensors
    depth_averaged = soil_moisture_masked.groupby('depth_group').mean(dim='sensor', skipna=True)
    depth_averaged = depth_averaged.rename({'depth_group': 'depth'})
    
    # Drop depths that have ALL NaN values (no valid data)
    valid_count_per_depth = depth_averaged.count(dim='date_time')
    depths_with_data = valid_count_per_depth > 0
    depth_averaged = depth_averaged.where(depths_with_data, drop=True)
    
    # If no valid depths remain, return None
    if len(depth_averaged.depth) == 0:
        return None
    
    # Resample to daily
    daily = depth_averaged.resample(date_time='1D').mean(dim='date_time', skipna=True)
    
    # Count valid observations per day
    count = depth_averaged.resample(date_time='1D').count(dim='date_time')
    
    # Handle the casting more gracefully
    try:
        count = count.fillna(0).astype(int)
    except:
        count = count.astype(float)
    
    # Mask out days with < 6 valid observations
    daily_filtered = daily.where(count >= 6)
    
    # Create dataset with both soil moisture and count
    result_ds = xr.Dataset({
        'soil_moisture': daily_filtered,
        'observation_count': count
    })
    
    # Add metadata as attributes
    result_ds.attrs['network'] = network
    result_ds.attrs['station'] = station
    result_ds.attrs['latitude'] = float(station_data.attrs.get('latitude', np.nan))
    result_ds.attrs['longitude'] = float(station_data.attrs.get('longitude', np.nan))
    
    return result_ds

## 'process_single_station' is just a wrapper for parallel processing. 
What really happens is:
1. station data is read into xarray. 
2. then process_station is called, which does QC, filtering, converts hourly to daily, which returns result_ds. 
3. If result_ds is None there we skip to other stations, if not
4. We get metadata like network, stations, longitude, latitude, depths, start_date, end_date, etc.
5. Filename is saved as f"{network}_{station}_{start_date}_{end_date}.nc" in output_dir.
6. Function returns metadata.


In [ ]:
def process_single_station(args):
    """
    Wrapper function for parallel processing
    """
    network, station, idx, total = args
    
    try:
        print(f"[{idx+1}/{total}] Processing: {network}/{station}")
        
        # Read station data
        station_data = ds[network][station].to_xarray()
        
        # Process station
        result_ds = process_station(station_data, network, station)
        
        # Skip if no valid data
        if result_ds is None:
            print(f"[{idx+1}/{total}] Skipped (no valid data): {network}/{station}")
            return None
        
        # Get metadata
        lat = result_ds.attrs.get('latitude', np.nan)
        lon = result_ds.attrs.get('longitude', np.nan)
        depths = result_ds.depth.values.tolist()
        
        # Get date range
        valid_dates = result_ds['soil_moisture'].dropna(dim='date_time', how='all').date_time
        if len(valid_dates) == 0:
            print(f"[{idx+1}/{total}] Skipped (no valid dates): {network}/{station}")
            return None
            
        start_date = pd.to_datetime(valid_dates.min().values).strftime('%Y%m%d')
        end_date = pd.to_datetime(valid_dates.max().values).strftime('%Y%m%d')
        
        # Create filename
        filename = f"{network}_{station}_{start_date}_{end_date}.nc"
        filepath = output_dir / filename
        
        # Save to netCDF
        result_ds.to_netcdf(filepath)
        
        print(f"[{idx+1}/{total}] Success: {network}/{station} -> {filename}")
        
        # Return metadata
        metadata = {
            'network': network,
            'station': station,
            'latitude': lat,
            'longitude': lon,
            'depths': str(depths),  # Convert list to string for CSV
            'n_depths': len(depths),
            'start_date': start_date,
            'end_date': end_date,
            'n_days': len(valid_dates),
            'filename': filename
        }
        
        return metadata
        
    except Exception as e:
        print(f"[{idx+1}/{total}] Error processing {network}/{station}: {e}")
        return None


In [ ]:
# # Inspect the data structure
# print("Data structure:")
# print(ds_test)
# print("\n" + "="*60 + "\n")

# # Check dimensions
# print("Dimensions:")
# print(ds_test.dims)
# print("\n" + "="*60 + "\n")

# # Check coordinates
# print("Coordinates:")
# print(ds_test.coords)
# print("\n" + "="*60 + "\n")


In [14]:
# Collect all station tasks
all_tasks = []
for network in ds.collection.networks:
    for station in ds.collection[network].stations:
        all_tasks.append((network, station))

In [ ]:

# ============================================
# CONFIGURE TEST RUN HERE
# ============================================
TEST_MODE = True  # Set to False to process all stations
N_TEST_STATIONS = 1  # Number of stations to test with

if TEST_MODE:
    tasks = all_tasks[:N_TEST_STATIONS]
    print(f"=== TEST MODE: Processing {N_TEST_STATIONS} stations ===")
else:
    tasks = all_tasks
    print(f"=== FULL MODE: Processing all {len(tasks)} stations ===")

# Add index and total count to tasks
tasks_with_idx = [(net, sta, i, len(tasks)) for i, (net, sta) in enumerate(tasks)]

print(f"Output directory: {output_dir}/")
print(f"Using {cpu_count()} CPU cores available")

=== TEST MODE: Processing 10 stations ===
Output directory: processed_soil_moisture/
Using 128 CPU cores available


In [ ]:

# ============================================
# CONFIGURE PARALLELIZATION HERE
# ============================================
USE_PARALLEL = False  # Set to False for sequential (easier debugging)
N_WORKERS = 10  # Number of parallel workers (adjust as needed)

if USE_PARALLEL:
    print(f"Running in parallel with {N_WORKERS} workers")
    with Pool(N_WORKERS) as pool:
        results = pool.map(process_single_station, tasks_with_idx)
else:
    print("Running sequentially (no parallelization)")
    results = [process_single_station(task) for task in tasks_with_idx]

Running in parallel with 10 workers
[2/10] Processing: AMMA-CATCH/Belefoungou-Mid[5/10] Processing: AMMA-CATCH/Nalohou-Top[1/10] Processing: AMMA-CATCH/Banizoumbou[3/10] Processing: AMMA-CATCH/Belefoungou-Top[9/10] Processing: ARM/Ashton[4/10] Processing: AMMA-CATCH/Nalohou-Mid[10/10] Processing: ARM/Byron[6/10] Processing: AMMA-CATCH/Tondikiboro[7/10] Processing: AMMA-CATCH/Wankama[8/10] Processing: ARM/Anthony









[4/10] Success: AMMA-CATCH/Nalohou-Mid -> AMMA-CATCH_Nalohou-Mid_20140101_20171231.nc[1/10] Success: AMMA-CATCH/Banizoumbou -> AMMA-CATCH_Banizoumbou_20140101_20181231.nc

[5/10] Success: AMMA-CATCH/Nalohou-Top -> AMMA-CATCH_Nalohou-Top_20140101_20171231.nc
[3/10] Success: AMMA-CATCH/Belefoungou-Top -> AMMA-CATCH_Belefoungou-Top_20140101_20171231.nc
[2/10] Success: AMMA-CATCH/Belefoungou-Mid -> AMMA-CATCH_Belefoungou-Mid_20140101_20171110.nc
[6/10] Success: AMMA-CATCH/Tondikiboro -> AMMA-CATCH_Tondikiboro_20140101_20181231.nc
[7/10] Success: AMMA-CATCH/Wankama -> AMMA-

In [18]:

# Filter out None results
metadata_list = [r for r in results if r is not None]

# Create metadata DataFrame
if len(metadata_list) > 0:
    df_metadata = pd.DataFrame(metadata_list)
    
    print(f"\n{'='*60}")
    print(f"Processing complete!")
    print(f"Successfully processed: {len(df_metadata)}/{len(tasks)} stations")
    print(f"Files saved to: {output_dir}/")
    print(f"\nMetadata summary:")
    print(df_metadata.head(20))
    
    # Save metadata
    metadata_file = output_dir / 'station_metadata.csv'
    df_metadata.to_csv(metadata_file, index=False)
    print(f"\nMetadata saved to: {metadata_file}")
    
    # Print summary statistics
    print(f"\nSummary:")
    print(f"  Networks: {df_metadata['network'].nunique()}")
    print(f"  Stations: {len(df_metadata)}")
    print(f"  Date range: {df_metadata['start_date'].min()} to {df_metadata['end_date'].max()}")
    print(f"  Depth range: {df_metadata['n_depths'].min()}-{df_metadata['n_depths'].max()} depths per station")
    
else:
    print("\nNo valid data found across all stations")


Processing complete!
Successfully processed: 10/10 stations
Files saved to: processed_soil_moisture/

Metadata summary:
      network          station  latitude  longitude  \
0  AMMA-CATCH      Banizoumbou       NaN        NaN   
1  AMMA-CATCH  Belefoungou-Mid       NaN        NaN   
2  AMMA-CATCH  Belefoungou-Top       NaN        NaN   
3  AMMA-CATCH      Nalohou-Mid       NaN        NaN   
4  AMMA-CATCH      Nalohou-Top       NaN        NaN   
5  AMMA-CATCH      Tondikiboro       NaN        NaN   
6  AMMA-CATCH          Wankama       NaN        NaN   
7         ARM          Anthony       NaN        NaN   
8         ARM           Ashton       NaN        NaN   
9         ARM            Byron       NaN        NaN   

                                              depths  n_depths start_date  \
0                                             [0.05]         1   20140101   
1                    [0.05, 0.1, 0.2, 0.4, 0.6, 1.0]         6   20140101   
2                         [0.05, 0.1, 0.4,

In [20]:
metadata_list

[{'network': 'AMMA-CATCH',
  'station': 'Banizoumbou',
  'latitude': nan,
  'longitude': nan,
  'depths': '[0.05]',
  'n_depths': 1,
  'start_date': '20140101',
  'end_date': '20181231',
  'n_days': 1824,
  'filename': 'AMMA-CATCH_Banizoumbou_20140101_20181231.nc'},
 {'network': 'AMMA-CATCH',
  'station': 'Belefoungou-Mid',
  'latitude': nan,
  'longitude': nan,
  'depths': '[0.05, 0.1, 0.2, 0.4, 0.6, 1.0]',
  'n_depths': 6,
  'start_date': '20140101',
  'end_date': '20171110',
  'n_days': 1400,
  'filename': 'AMMA-CATCH_Belefoungou-Mid_20140101_20171110.nc'},
 {'network': 'AMMA-CATCH',
  'station': 'Belefoungou-Top',
  'latitude': nan,
  'longitude': nan,
  'depths': '[0.05, 0.1, 0.4, 0.6, 1.0]',
  'n_depths': 5,
  'start_date': '20140101',
  'end_date': '20171231',
  'n_days': 1448,
  'filename': 'AMMA-CATCH_Belefoungou-Top_20140101_20171231.nc'},
 {'network': 'AMMA-CATCH',
  'station': 'Nalohou-Mid',
  'latitude': nan,
  'longitude': nan,
  'depths': '[0.05, 0.1, 0.2, 0.4]',
  'n_de

In [27]:
df_metadata = pd.read_csv('/home/khanalp/code/PhD/soilMoisture/processed_soil_moisture/station_metadata.csv')

In [29]:
df_metadata

,network,station,latitude,longitude,depths,n_depths,start_date,end_date,n_days,filename
0,AMMA-CATCH,Banizoumbou,NaN,NaN,[0.05],1,20140101,20181231,1824,AMMA-CATCH_Banizoumbou_20140101_20181231.nc
1,AMMA-CATCH,Belefoungou-Mid,NaN,NaN,"[0.05, 0.1, 0.2, 0.4, 0.6, 1.0]",6,20140101,20171110,1400,AMMA-CATCH_Belefoungou-Mid_20140101_20171110.nc
2,AMMA-CATCH,Belefoungou-Top,NaN,NaN,"[0.05, 0.1, 0.4, 0.6, 1.0]",5,20140101,20171231,1448,AMMA-CATCH_Belefoungou-Top_20140101_20171231.nc
3,AMMA-CATCH,Nalohou-Mid,NaN,NaN,"[0.05, 0.1, 0.2, 0.4]",4,20140101,20171231,1395,AMMA-CATCH_Nalohou-Mid_20140101_20171231.nc
4,AMMA-CATCH,Nalohou-Top,NaN,NaN,"[0.05, 0.1, 0.2, 0.6, 1.0]",5,20140101,20171231,1330,AMMA-CATCH_Nalohou-Top_20140101_20171231.nc
5,AMMA-CATCH,Tondikiboro,NaN,NaN,"[0.05, 0.1, 0.4, 0.7, 1.05]",5,20140101,20181231,1822,AMMA-CATCH_Tondikiboro_20140101_20181231.nc
6,AMMA-CATCH,Wankama,NaN,NaN,"[0.05, 0.1, 0.4, 0.7, 1.0]",5,20140101,20181231,1826,AMMA-CATCH_Wankama_20140101_20181231.nc
7,ARM,Anthony,NaN,NaN,"[0.025, 0.05, 0.1, 0.2, 0.5, 0.8]",6,20140113,20210921,2622,ARM_Anthony_20140113_20210921.nc
8,ARM,Ashton,NaN,NaN,"[0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.5, 0.6, 0...",9,20140101,20230929,2849,ARM_Ashton_20140101_20230929.nc
9,ARM,Byron,NaN,NaN,"[0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.5, 0.6, 0...",11,20140101,20230929,2803,ARM_Byron_20140101_20230929.nc


In [ ]:
# Create mask where soil_moisture_flag == "G"
mask = ds_test['soil_moisture_flag'] == "G"
print(f"\nNumber of 'G' flags: {mask.sum().values}")

In [ ]:
# Apply mask to soil_moisture
soil_moisture_masked = ds_test['soil_moisture'].where(mask)

# Check unique depths
print(f"\nUnique depths: {np.unique(ds_test['depth_from'].values)}")
    

In [ ]:
# Assign depth_from as a coordinate for grouping
soil_moisture_masked = soil_moisture_masked.assign_coords(
    depth_group=('sensor', ds_test['depth_from'].values)
)


In [ ]:
# Group by depth and average across sensors
depth_averaged = soil_moisture_masked.groupby('depth_group').mean(dim='sensor', skipna=True)
depth_averaged = depth_averaged.rename({'depth_group': 'depth'})

In [ ]:
depth_averaged

In [ ]:
# Resample to daily
daily = depth_averaged.resample(date_time='1D').mean(dim='date_time', skipna=True)

# Count valid observations per day - ensure it's numeric
count = depth_averaged.resample(date_time='1D').count(dim='date_time')

In [ ]:
# Convert count to int if needed
count = count.astype(int)

# Mask out days with < 6 valid observations
daily_filtered = daily.where(count >= 6)

In [ ]:
daily_filtered.plot()

In [ ]:
trial = xr.open_dataset('/home/khanalp/code/PhD/soilMoisture/processed_soil_moisture/ARM_Byron_20140101_20230929.nc')

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

# Load the dataset
trial = xr.open_dataset('/home/khanalp/code/PhD/soilMoisture/processed_soil_moisture/AMMA-CATCH_Wankama_20140101_20181231.nc')

# Plot all depths on one plot (overlapping)
fig, ax = plt.subplots(figsize=(14, 6))

for depth in trial.depth.values:
    trial['soil_moisture'].sel(depth=depth).plot(ax=ax, label=f'{depth} m')

ax.set_ylabel('Soil Moisture')
ax.set_xlabel('Date')
ax.set_title(f'Soil Moisture - {trial.attrs["station"]} ({trial.attrs["network"]})')
ax.legend(title='Depth')
ax.grid(True, alpha=0.3)
plt.tight_layout()
# plt.savefig('soil_moisture_by_depth.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
trial